In [ ]:
%run dataset.ipynb
%run models.ipynb
import time
import random
import numpy as np
from glob import glob
from tqdm import tqdm
from operator import add
from PIL import Image
import segmentation_models_pytorch as smp
import os 

import re
import torch.optim as optim
                    
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
import import_ipynb
import json
from pyfiles.efficientunet import *

In [ ]:

# Load the JSON file
with open('dict_indices.json', 'r') as file:
    data = json.load(file)
train_indices = data['train_indices']
val_indices = data['val_indices']

with open('domainb_indices.json', 'r') as file:
    data = json.load(file)

domainb_indices = data['train']


train_path = os.path.join(os.getcwd(),"train")
mask_path = os.path.join(os.getcwd(),"train_gt")
train_x = os.listdir('train')
train_x.sort(key=extract_numerical_part)
train_y = os.listdir('train_gt')
train_y.sort(key=extract_numerical_part)

val_path = os.path.join(os.getcwd(),"val")
val_gt = os.path.join(os.getcwd(),"val_gt")
val_x = os.listdir('val')
val_x.sort(key=extract_numerical_part)
val_y = os.listdir('val_gt')
val_y.sort(key=extract_numerical_part)

testB = os.path.join(os.getcwd(),"imgs")
test_gtB = os.path.join(os.getcwd(),"Gts")
path_soft_gtB  =  os.path.join(os.getcwd(),"soft_gtB")

testB_images = os.listdir(testB)
testB_images.sort(key=extract_numerical_part)
testB_masks = os.listdir(test_gtB)
testB_masks.sort(key=extract_numerical_part)



# vss = dess400(train_path,mask_path,train_indices,train_transform)
# vall = dess400(val_path,val_gt,val_indices,valid_transform)
# dbb = dess400(testB,test_gtB,domainb_indices,train_transform)

vs= Dess2(train_path,mask_path,train_x ,train_y , train_transform)

val= Dess2(val_path , val_gt, val_x,val_y , valid_transform)
domainb = hotencoded(testB , test_gtB , testB_images    , testB_masks , num_classes=7,transform=valid_transform)
# domainb_soft = hot400(testB , path_soft_gtB, domainb_indices, num_classes=7,transform=train_transform)
domainb_small = hot400(testB , test_gtB,domainb_indices,num_classes=7,transform=valid_transform)
ds = hot400(train_path , mask_path,train_indices,num_classes=7,transform=train_transform)
ds_val = hot400(val_path , val_gt , val_indices,num_classes=7,transform=valid_transform)
# ds = hotencoded(train_path,mask_path , train_x,train_y , num_classes=7,transform=train_transform)
# ds_val = hotencoded(val_path,val_gt,val_x,val_y,num_classes=7,transform=valid_transform)
dbtrain,dbtest = splitset(testB_masks,0.8)
dbtrain_l,dbtrain_ul = splitset(dbtrain,0.25)
domainb_train_l = hot400(testB , test_gtB,dbtrain_l,num_classes=7,transform=train_transform)
domainb_train_ul = hot400(testB , test_gtB,dbtrain_ul,num_classes=7,transform=train_transform)
domainb_test = hot400(testB,test_gtB , dbtest,num_classes=7,transform=valid_transform)
db_whole = hot400(testB,test_gtB,testB_masks,num_classes=7,transform=valid_transform)


In [ ]:
# Define the label_colors
label_colors = [
    [0, 0, 0],       # Background (black)
    [255, 0, 0],     # Class 1 (red)
    [0, 255, 0],     # Class 2 (green)
    [0, 0, 255],     # Class 3 (blue)
    [255, 255, 0],   # Class 4 (yellow)
    [255, 0, 255],   #Class 5 (magenta)
    [0, 255, 255]    # Class 6 (cyan)
]

# Set the directory where the folders are located
dir = '/home/khan/Desktop/ammar/DataAugustFull/OAI-MRI-DESS/save_masks/rawpredictions'

# Get the list of folders
ll = ['Gts', '83.605', '75.41', '72.638']

# Get the list of image files starting from index 150
image_files = [f'{i+1}.png' for i in range(3150, 3300)]

# Set the image size for each subplot
image_size = (800, 800)

# Create the figure and axes
fig, axes = plt.subplots(len(image_files), len(ll), figsize=(16, len(image_files) * 4), gridspec_kw={'wspace': 0.05, 'hspace': 0.05})

# Iterate over the image files
for i, image_file in enumerate(image_files):
    # Iterate over the folders
    for j, folder in enumerate(ll):
        folder_path = os.path.join(dir, folder)
        image_path = os.path.join(folder_path, image_file)
        
        # Load the image
        mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Create the color mask
        color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
        for label in range(7):
            color_mask[mask == label] = label_colors[label]
        
        # Resize the color mask to the desired size
        color_mask = cv2.resize(color_mask, image_size)
        
        # Plot the color mask
        axes[i, j].imshow(color_mask)
        axes[i, j].set_title(f'{folder}: {image_file}', fontsize=8)
        axes[i, j].text(0, -0.2, f'Filename: {image_file}', transform=axes[i, j].transAxes, fontsize=10)
        axes[i, j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
np.unique(mask)

In [ ]:
#show predictions for saved model masks 
# label_colors = [
#     [0, 0, 0],       # Background (black)
#     [0, 0, 0],     # Class 1 (red)
#     [0, 255, 0],     # Class 2 (green)
#     [0, 0, 255],     # Class 3 (blue)
#     [255, 255, 0],   # Class 4 (yellow)
#     [255, 0, 255],   # Class 5 (magenta)
#     [0, 255, 255]    # Class 6 (cyan)
# ]

label_colors = [
    [0, 0, 0],       # Background (black)
    [0, 0, 0],     # Class 1 (red)
    [0, 0, 0],     # Class 2 (green)
    [0, 0, 255],     # Class 3 (blue)
    [0,0, 0],   # Class 4 (yellow)
    [255, 0, 255],   # Class 5 (magenta)
    [0, 0, 0]    # Class 6 (cyan)
]


# Set the directory where the folders are located
dir = '/home/khan/Desktop/ammar/DataAugustFull/OAI-MRI-DESS/save_masks/rawpredictions'

# Get the list of folders
ll = ['Gts', '83.605', '75.41', '72.638']

# Define the list of image file names to show
image_file_names = ['3270.png',  '3201.png' ]

# Set the image size for each subplot
image_size = (800, 800)

# Create the figure and axes
fig, axes = plt.subplots(len(image_file_names), len(ll), figsize=(16, len(image_file_names) * 4), gridspec_kw={'wspace': 0.05, 'hspace': 0.05})

# Iterate over the image file names
for i, image_file in enumerate(image_file_names):
    # Iterate over the folders
    for j, folder in enumerate(ll):
        folder_path = os.path.join(dir, folder)
        image_path = os.path.join(folder_path, image_file)
        
        # Load the image
        mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) 
        
        # Create the color mask
        color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
        for label in range(7):
            color_mask[mask == label] = label_colors[label]
        
        # Resize the color mask to the desired size
        color_mask = cv2.resize(color_mask, image_size)
        
        # Plot the color mask
        axes[i, j].imshow(color_mask)
        axes[i, j].set_title(f'{folder}: {image_file}', fontsize=8)
        axes[i, j].text(0, -0.2, f'Filename: {image_file}', transform=axes[i, j].transAxes, fontsize=10)
        axes[i, j].axis('off')

plt.tight_layout()
# plt.show()
# Save the collage as an image file
collage_filename = '3_5_combo.png'
plt.savefig(collage_filename, dpi=300, bbox_inches='tight')
print(f'Collage saved as {collage_filename}')

In [ ]:
# Set the directory where the folders are located
dir = '/home/khan/Desktop/ammar/DataAugustFull/OAI-MRI-DESS/save_masks/rawpredictions'

# Get the list of folders
ll = ['Gts', '83.605', '75.41', '72.638']

# Define the list of image file names to save
image_file_names = ['3241.png', '3248.png', '3204.png', '3213.png', '3251.png', '3246.png', '3270.png','3201.png']

# Create the 'shortlisted' directory if it doesn't exist
shortlisted_dir = 'shortlisted2'
if not os.path.exists(shortlisted_dir):
    os.makedirs(shortlisted_dir)

# Iterate over the folders
for folder in ll:
    folder_path = os.path.join(dir, folder)
    
    # Iterate over the image file names
    for image_file in image_file_names:
        image_path = os.path.join(folder_path, image_file)
        
        # Load the original image
        original_image = cv2.imread(image_path)
        
        # Create the new file name with the folder name and image name
        new_filename = f"{folder}_{image_file}"
        new_file_path = os.path.join(shortlisted_dir, new_filename)
        
        # Save the original image to the 'shortlisted' directory
        cv2.imwrite(new_file_path, original_image)

print("Original images saved in the 'shortlisted' directory.")

In [ ]:
imgs = cv2.imread("/home/khan/Desktop/ammar/DataAugustFull/OAI-MRI-DESS/save_masks/rawpredictions/53.66/2500.png",cv2.IMREAD_GRAYSCALE)
# plt.imshow('/home/khan/Desktop/ammar/DataAugustFull/OAI-MRI-DESS/save_masks/rawpredictions/53.66/1.png')


dir = os.listdir('/home/khan/Desktop/ammar/DataAugustFull/OAI-MRI-DESS/save_masks/rawpredictions')
ll=['Gts','83.605' , '75.41' , '62.36']


color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
for label in range(7):  # Iterate over labels 0 to 6
    color_mask[ == label] = label_colors[label]

In [ ]:
plt.imshow(imgs)


In [ ]:
plt.imshow(color_mask)

In [ ]:
#display only groundtruth
import matplotlib.pyplot as plt
import numpy as np

# # Define the label colors
label_colors = [
    [0, 0, 0],       # Background (black)
    [255, 0, 0],     # Class 1 (red)
    [0, 255, 0],     # Class 2 (green)
    [0, 0, 255],     # Class 3 (blue)
    [255, 255, 0],   # Class 4 (yellow)
    [255, 0, 255],   #Class 5 (magenta)
    [0, 255, 255]    # Class 6 (cyan)
]

# Create a color mask for the entire dataset
count = 0
for i in range(len(domainb)):
    count = count+1
    # if count==150:
    #     break
    image, _,mask, filename = domainb_test[i]
    if int(filename.split('.')[0]) in shorts:
        # Plot the input image, transformed image, transformed mask, and color mask
        fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    
        
    
        axes[0].imshow(image.permute(1,2,0))
        axes[0].set_title('Transformed Image')
        axes[0].axis('off')
    
        # Create a color mask by mapping the label values to colors
        color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
        for label in range(7):  # Iterate over labels 0 to 6
            color_mask[mask == label] = label_colors[label]
    
        axes[1].imshow(color_mask)
        axes[1].set_title('Color Mask')
        axes[1].axis('off')
    
        axes[1].text(0, -0.2, f'Filename: {filename}', transform=axes[1].transAxes, fontsize=12)
    
        plt.tight_layout()
        plt.show()




In [ ]:
from pyfiles.efficientunet import *


In [ ]:
# ##to display and save one single image prediction
# # import matplotlib.pyplot as plt
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Define the label colors (you can change the variable name to something else)
# class_colors = [
#         [255, 0, 0],     # Class 1 (red)
#         [0, 255, 0],     # Class 2 (green)
#         [0, 0, 255],     # Class 3 (blue)
#         [255, 255, 0],   # Class 4 (yellow)
#         [255, 0, 255],   # Class 5 (magenta)
#         [0, 255, 255]    # Class 6 (cyan)
#     ]
# num_classes=7
# # Iterate through each image in the dataset

    
# student_model.eval()       
# with torch.no_grad():
#     output = student_model(imgss.float().unsqueeze(0))
# pred = torch.argmax(output, dim=1).squeeze(0)
# pred = pred.to("cpu").numpy()

# #f1 score calculation 
# f1_scores = np.zeros(num_classes)
# for class_label in range(num_classes):  # F1 score and loss for the current class
#     pred_class = (pred == class_label)
#     real_class = (mk == class_label)
    
#     true_positives = np.sum(np.logical_and(pred_class, real_class).numpy())
#     false_positives = np.sum(np.logical_and(pred_class, ~real_class).numpy())
#     false_negatives = np.sum(np.logical_and(~pred_class, real_class).numpy())
    
#     precision = true_positives / (true_positives + false_positives)
#     recall = true_positives / (true_positives + false_negatives)
    
#     f1_scores[class_label] = 2 * (precision * recall) / (precision + recall)

# original_image = imgss.permute(1, 2, 0).numpy()
# segmentation_mask = mk.numpy()
# pred_mask = pred


# # Create a copy of the original image for visualization
# mapped_image = original_image.copy()
# pred_img = original_image.copy()

# # Apply color or overlay to each segmented region based on labels
# for label in range(1, np.max(segmentation_mask) + 1):
#     if label -1< len(class_colors):  # Check if a color is defined for the label
#         region_pixels = np.where(segmentation_mask == label)
#         mapped_image[region_pixels] = class_colors[label - 1]
         
# for label in range(1, np.max(pred_img) + 1):
#     if label-1 < len(class_colors):  # Check if a color is defined for the label
#         region_pixels = np.where(pred_mask == label)
#         pred_img[region_pixels] = class_colors[label - 1]
    
# # Create subplots for original image, ground truth mask, and predicted mask
# fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# # Plot the original image
# axs[0].imshow(original_image)
# axs[0].set_title("Original Image")

# # Plot the ground truth mask with mapped color labels
# axs[1].imshow(mapped_image)
# axs[1].set_title("Ground Truth Mask")

# # Plot the predicted mask mapped onto the original image
# axs[2].imshow(pred_img)
# axs[2].set_title("Predicted Mask")

# # Add the caption with the name variable
# axs[2].text(0.5, -0.1, m, transform=axs[2].transAxes, ha='center')

# # Remove axis labels
# for ax in axs:
#     ax.axis("off")

# # # Save the figure to a local directory
# # # pred_img_bgr = pred_img[..., ::-1]

# # save_dir = '/home/khan/Desktop/ammar/DataAugustFull/OAI-MRI-DESS/partitions/3126'
# # os.makedirs(save_dir, exist_ok=True)
# # img_name = os.path.join(save_dir,model_name)
# # plt.imsave(img_name,pred_img)
        



In [ ]:
['efb4-full.png','studentSW_efb4_257520.png','student_efb4_257520.png','resnet34.png'] #api models 
['studentsvdSW3_efb4_ponit05.png','studentsvdSW_efb4_257520.png','studentsvdSW2_efb4_257520.png','newefb4svd-full.pth',
'studentTsvd_efb4_257520.png', '2_efb4_ponit05.png', 'studentsvdSW3_efb4_356520.pth'] #sutombackbones 
[fivepercent ]
#percentages
[28 , 81.33,0.5, 25 ] #api models 

[52.89, 84.23 , 85.07,44.72, 82.50,77.64]
[65.78] #five percent 




In [ ]:
device="cuda"

model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=7,                      # model output channels (number of classes in your dataset)
)
old_teacher_model = smp.Unet(
    encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=7,                      # model output channels (number of classes in your dataset)
)
student_model = get_efficientunet_b4(out_channels=7, concat_input=True, pretrained=True)
# old_teacher_model.load_state_dict(torch.load("2_efb4_ponit05.pth"))

In [ ]:
model.load_state_dict(torch.load('resnet34.pth'))


In [ ]:
loss_fn = CategoricalFocalLoss(gamma=2.0, alpha=None, reduction='mean')
num_classes=7


In [ ]:

student_model=model

In [ ]:


#0,2,4,

student_model.load_state_dict(torch.load('studentsvdSW2_efb4_257520.pth'))


In [ ]:
student_model = student_model.to(device)
l,t,tt= validlossf2(student_model,domainb_test,loss_fn,num_classes)
t

In [ ]:
l,t,tt,df = validlossJ(student_model, domainb_test, loss_fn , num_classes)

In [ ]:
np.unique(cv2.imread('/home/khan/Desktop/ammar/DataAugustFull/OAI-MRI-DESS/save_masks/rawpredictions/62.36/200.png',cv2.IMREAD_GRAYSCALE))

In [ ]:
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the label colors (you can change the variable name to something else)
class_colors = [
        [255, 0, 0],     # Class 1 (red)
        [0, 255, 0],     # Class 2 (green)
        [0, 0, 255],     # Class 3 (blue)
        [255, 255, 0],   # Class 4 (yellow)
        [255, 0, 255],   # Class 5 (magenta)
        [0, 255, 255]    # Class 6 (cyan)
    ]

# Iterate through each image in the dataset
student_model.eval()
for image, _, mask, name in domainb:
    image=image.to(device)
    mask = mask.to(device)
    
    # if int(name.split('.')[0]) in pics:
    
    student_model.to(device)    
    with torch.no_grad():
       
        result = student_model(image.float().unsqueeze(0))
        predictions = torch.argmax(result, dim=1)
        pred_labels = predictions.squeeze().cpu().numpy()
    
    original_image = image.permute(1, 2, 0).cpu().numpy()
    segmentation_mask = mask.cpu().numpy()
    pred_mask = pred_labels

        
    # Create a copy of the original image for visualization
    mapped_image = original_image.copy()
    pred_img = original_image.copy()

    for label in range(1, np.max(segmentation_mask) + 1):
        if label -1< len(class_colors):  # Check if a color is defined for the label
            region_pixels = np.where(segmentation_mask == label)
            mapped_image[region_pixels] = class_colors[label - 1]
     
    for label in range(1, np.max(pred_img) + 1):
        if label-1 < len(class_colors):  # Check if a color is defined for the label
            region_pixels = np.where(pred_mask == label)
            pred_img[region_pixels] = class_colors[label - 1]

    # Create subplots for original image, ground truth mask, and predicted mask
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))

    # Plot the original image
    axs[0].imshow(original_image)
    axs[0].set_title("Original Image")

    # Plot the ground truth mask with mapped color labels
    axs[1].imshow(mapped_image)
    axs[1].set_title("Ground Truth Mask")

    # Plot the predicted mask mapped onto the original image
    axs[2].imshow(pred_img)
    axs[2].set_title("Predicted Mask")

    # Add the caption with the name variable
    axs[2].text(0.5, -0.1, name, transform=axs[2].transAxes, ha='center')

    # Remove axis labels
    for ax in axs:
        ax.axis("off")

    # Save the figure to a local directory
    # pred_img_bgr = pred_img[..., ::-1]

#     save_dir = '/home/khan/Desktop/ammar/DataAugustFull/OAI-MRI-DESS/save_masks/domainb_rawpredictions/62.36'
#     os.makedirs(save_dir, exist_ok=True)
#     img_name = os.path.join(save_dir,name)
#     # plt.imsave(img_name,pred_labels)
#     cv2.imwrite(img_name,pred_mask.astype(np.uint8))
# print("ALL SAVED")
    # # Show the subplots
    plt.tight_layout()
plt.show()
plt.close()


In [ ]:
def validlossJ(model, ds, loss_fn, num_classes):
    jaccard_list = []
    loss_list = []
    
    model.eval()
    for img, mask, orig, _ in ds:
        img = img.to(device)
        mask = mask.to(device)
        with torch.no_grad():
            output = model(img.float().unsqueeze(0))
        loss = loss_fn(output, mask.float().unsqueeze(0))
        pred = torch.argmax(output, dim=1).squeeze(0)
        pred = pred.to("cpu")
        
        jaccard_scores = np.zeros(num_classes)
        
        for class_label in range(num_classes):  # Jaccard Index and loss for the current class
            pred_class = (pred == class_label)
            real_class = (orig == class_label)
            
            intersection = np.sum(np.logical_and(pred_class, real_class).numpy())
            union = np.sum(np.logical_or(pred_class, real_class).numpy())
            
            jaccard_scores[class_label] = intersection / union
        
        jaccard_list.append(np.round(list(jaccard_scores), 5))
        loss_list.append(loss.item())
      
    average_loss = np.average(loss_list)
    jaccard_df = pd.DataFrame(jaccard_list)
    jaccard_avg = jaccard_df[jaccard_df != 0].mean()
    average_jaccard = np.round(np.average(jaccard_avg), 5)

    return np.round(average_loss, 7), np.round(average_jaccard, 5), np.round(jaccard_avg, 5), jaccard_df

def validlossf3(model, ds, loss_fn, num_classes):
    f1_list = []
    loss_list = []
    
    model.eval()
    for img, mask, orig, _ in ds:
        img = img.to(device)
        mask = mask.to(device)
        with torch.no_grad():
            output = model(img.float().unsqueeze(0))
        loss = loss_fn(output, mask.float().unsqueeze(0))
        pred = torch.argmax(output, dim=1).squeeze(0)
        pred = pred.to("cpu")
        
        f1_scores = np.zeros(num_classes)
        
        for class_label in range(num_classes):  # F1 score and loss for the current class
            pred_class = (pred == class_label)
            real_class = (orig == class_label)
            
            true_positives = np.sum(np.logical_and(pred_class, real_class).numpy())
            false_positives = np.sum(np.logical_and(pred_class, ~real_class).numpy())
            false_negatives = np.sum(np.logical_and(~pred_class, real_class).numpy())
            
            precision = true_positives / (true_positives + false_positives)
            recall = true_positives / (true_positives + false_negatives)
            
            f1_scores[class_label] = 2 * (precision * recall) / (precision + recall)
        
        f1_list.append(np.round(list(f1_scores),5))
        loss_list.append(loss.item())
      
    average_loss = np.average(loss_list)
    f1df = pd.DataFrame(f1_list)
    f1avg = f1df[f1df != 0].mean()
    average_f1 = np.round(np.average(f1avg),5)

    return np.round(average_loss,7), np.round(average_f1,5), np.round(f1avg,5),f1df

In [ ]:
domain

In [ ]:
fdict={} 
f1_scores=[]
for md in modelss:
    if md in ["efb4-full.pth",'studentSW_efb4_257520.pth','student_efb4_257520.pth','resnet34.pth']:
        pass
    else:
        student_model.load_state_dict(torch.load(md,map_location = torch.device(device)))
    
        with torch.no_grad():
            output = student_model(imgss.float().unsqueeze(0))
        pred = torch.argmax(output, dim=1).squeeze(0)
        pred = pred.to("cpu")
                
        f1_scores = np.zeros(num_classes)
        for class_label in range(num_classes):  # F1 score and loss for the current class
            pred_class = (pred == class_label)
            real_class = (mk == class_label)
            
            true_positives = np.sum(np.logical_and(pred_class, real_class).numpy())
            false_positives = np.sum(np.logical_and(pred_class, ~real_class).numpy())
            false_negatives = np.sum(np.logical_and(~pred_class, real_class).numpy())
            
            precision = true_positives / (true_positives + false_positives)
            recall = true_positives / (true_positives + false_negatives)
            
            f1_scores[class_label] = 2 * (precision * recall) / (precision + recall)
        fdict[md] = f1_scores[-1]
                



In [ ]:
pics = [2663, 2666,2745,2746,2755,2756,2773,2774,2786,2797,2810
,2811]

In [ ]:
len(pics)

In [ ]:
ll = os.listdir(os.path.join(dir,"sourcefree"))

In [ ]:
ll.sort()

In [ ]:
i,t,tt=validlossf1(student_model,domainb,loss_fn,num_classes=7)


In [ ]:
name

In [ ]:
dir = '/home/khan/Desktop/ammar/DataAugustFull/OAI-MRI-DESS/save_masks'
os.listdir(dir)

In [ ]:
os.listdir(os.path.join(dir,'studentSW_efb4_257520.pth_dbtest_80.88'))

In [ ]:
dir

In [ ]:
import os
import matplotlib.pyplot as plt

# Define the directory path where the images are located
directory_path = dir
# Define the list of folder names containing the images
folder_names = ['images','masks','resnet34.pth','2_efb4_ponit05.pth']
 

# Sort the folder names based on accuracy from highest to lowest
folder_names.sort(key=lambda x: float(x.split('_dbtest_')[1].split('.png')[0]) if '_dbtest_' in x else -1, reverse=True)

# Define the list of image names
image_names = [str(pic) + '.png' for pic in [2663, 2745, 2755, 2773, 2786, 2811]]

# Calculate the number of images per row
images_per_row = len(folder_names)

# Calculate the number of rows
num_rows = len(image_names)

# Set the figure size based on the number of images
fig_width = 4 * images_per_row
fig_height = 4 * num_rows
fig, axs = plt.subplots(num_rows, images_per_row, figsize=(fig_width, fig_height))

# Iterate over the image names and plot the corresponding images from each folder
for i, image_name in enumerate(image_names):
    for j, folder_name in enumerate(folder_names):
        # Get the path of the image in the current folder
        image_path = os.path.join(directory_path, folder_name, image_name)

        # Load and plot the image
        image = plt.imread(image_path)
        axs[i, j].imshow(image)
        axs[i, j].axis('off')

        # Add titles to the folder names for the first row
        if i == 0:
            axs[0, j].set_title(folder_name, fontsize=10)

# Remove spacing between subplots
plt.subplots_adjust(wspace=0, hspace=0)

# Show the grid of images
plt.show()

In [ ]:

import os
import matplotlib.pyplot as plt

# Define the directory path where the images are located
directory_path = dir



# Define the list of folder names containing the images
# Define the list of folder names containing the images in the desired order
folder_names = ['images','masks',
 'proposed',
 'cps',
 'fldkd',
 'mt',
'uda1','uda2', 'sourcefree',
 ]

# Define the list of image names in the desired order
image_names = ['2663.png', '2786.png', '2745.png', '2811.png', '2755.png', '2773.png']

# Calculate the number of images per row
images_per_row = len(folder_names)

# Calculate the number of rows
num_rows = len(image_names)

# Set the figure size based on the number of images
fig_width = 4 * images_per_row
fig_height = 4 * num_rows
fig, axs = plt.subplots(num_rows, images_per_row, figsize=(fig_width, fig_height))

# Iterate over the image names and plot the corresponding images from each folder
for i, image_name in enumerate(image_names):
    for j, folder_name in enumerate(folder_names):
        # Get the path of the image in the current folder
        image_path = os.path.join(directory_path, folder_name, image_name)

        # Load and plot the image
        image = plt.imread(image_path)
        axs[i, j].imshow(image)
        axs[i, j].axis('off')

# Remove spacing between subplots
fig.subplots_adjust(wspace=0, hspace=0)

# Remove any extra space around the figure
fig.tight_layout()

# Show the grid of images
plt.show()
plt.savefig('/path/to/save/grid_image.png')


In [ ]:
student_model.load_state_dict(torch.load("partitions/five_percent.pth"))

In [ ]:
student_model = student_model.to(device)
i,t,tt,df=validlossf3(student_model,domainb_test,loss_fn,num_classes=7)


In [ ]:
len(domainb_test)

In [ ]:
#___________________________________________________________________________________________________________________________

In [ ]:
a=torch.load("/home/khan/Desktop/ammar/EfficientUnet-PyTorch-master/ftmap")

In [ ]:
#we iterate over each channel of the feature map and plot it as a separate heatmap using the imshow function.
#The colorbar function is called to add a colorbar to each heatmap. This way, you can visualize each channel individually.
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have a feature map with dimensions (1792, 12, 12)
feature_map = a.to("cpu").detach().numpy()

# Normalize the feature map values between 0 and 1
normalized_map = (feature_map - feature_map.min()) / (feature_map.max() - feature_map.min())

# Iterate over the channels and plot them as heatmaps
for channel in range(feature_map.shape[0]):
    plt.imshow(normalized_map[channel], cmap='hot')
    plt.colorbar()
    plt.show()

In [ ]:
#visualize individual channels of the feature maps as grayscale images and display them side by side
# Normalize the feature map values between 0 and 1
normalized_map = (feature_map - feature_map.min()) / (feature_map.max() - feature_map.min())

# Select random channels to visualize
num_channels = 5  # Number of channels to visualize
selected_channels = np.random.choice(range(feature_map.shape[0]), num_channels, replace=False)

# Plot selected channels as grayscale images
fig, axes = plt.subplots(1, num_channels, figsize=(15, 3))

for i, channel in enumerate(selected_channels):
    axes[i].imshow(normalized_map[channel], cmap='gray')
    axes[i].set_title(f'Channel {channel}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Assuming you have a feature map with dimensions (1792, 12, 12)


# Stack the channels together to create a 3D volume
stacked_volume = np.stack(normalized_map, axis=-1)

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Set the x, y, z dimensions of the plot
x, y, z = np.meshgrid(range(stacked_volume.shape[0]), range(stacked_volume.shape[1]), range(stacked_volume.shape[2]))

# Plot the 3D volume
ax.scatter(x, y, z, c=stacked_volume.reshape(-1), cmap='hot')

# Set labels and title
ax.set_xlabel('Channel')
ax.set_ylabel('X')
ax.set_zlabel('Y')
ax.set_title('3D Visualization of Feature Maps')

# Show the plot
plt.show()

In [ ]:
b=torch.load("before")
af=torch.load("after")

In [ ]:
b.shape

In [ ]:
plt.imshow(a[100].detach().cpu().numpy())

In [ ]:
b=b.squeeze(0)

In [ ]:

b= torch.mean(b,dim=0)
# The result is a single channel with dimensions (width, height)
print(b.shape)

In [ ]:

af= torch.mean(af,dim=0)
# The result is a single channel with dimensions (width, height)
print(af.shape)

In [ ]:

plt.imshow(af)

In [ ]:
b=b.squeeze(0)

In [ ]:
# Reshape the feature map to have 10 feature maps per row
# Assuming you have a feature map of size torch.Size([1792, 12, 12])
feature_map = b
# Reshape the feature map to have 10 feature maps per row
# Reshape the feature map to have 10 feature maps per row
num_feature_maps = feature_map.shape[0]
num_rows = (num_feature_maps + 9) // 10
feature_map_grid = feature_map.view(num_rows, 10, feature_map.shape[1], feature_map.shape[2])

# Create the visualization grid
grid = utils.make_grid(feature_map_grid.flatten(0, 1), nrow=10, padding=1)

# Display the grid
plt.figure(figsize=(10, 10))
plt.imshow(grid.permute(1, 2, 0), cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
b.shape

In [ ]:
plt.imshow(b[0],cmap='grey')